In [8]:
import numpy as np  # library to handle data in a vectorized manner

import pandas as pd  # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.2f}'.format

import requests  # library to handle requests
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

import geocoder  # import geocoder
import plotly.express as px
import geopandas as gpd # to store geospatial data
import chart_studio
import chart_studio.plotly as py # for exporting Plotly visualizations to Chart Studio
import plotly.graph_objects as go # to ploty Plotly graph objects
import plotly.io as pio # Plotly renderer
import matplotlib.pyplot as plt # Import Matplotlib for visualizations
import datapane as dp # for exporting map visualizations to Datapane
from plotly.subplots import make_subplots # to make multiple Plotly plots in one instance

import plotly.offline as pyo # Set notebook mode to work in offline
pyo.init_notebook_mode()
from PyInstaller.utils.hooks import collect_dynamic_libs




print('Libraries imported.')

OSError: could not find or load spatialindex_c-64.dll

In [3]:
toronto_df = pd.read_excel(
    r'C:\Users\Glenn\Documents\GitHub\Finalproject\finalproject-data\CityofToronto_COVID-19_NeighbourhoodData.xlsx',
    sheet_name='All Cases and Rates by Neighbou') # Dataset with Neighbourhood names and ID
toronto_geo = r'C:\Users\Glenn\Documents\GitHub\Finalproject\finalproject-data\misc\Neighbourhoods.geojson'  # geojson file

print('Datasets downloaded')

Datasets downloaded


In [4]:

toronto_df.dropna(axis=0, inplace=True) # Drop empty rows
toronto_df = toronto_df.astype({"Neighbourhood ID": int}) # Convert Neighbourhood ID to Int type
toronto_df = toronto_df.iloc[:, 0:2] # Slice only relevant columns
toronto_df.head() # Display top 5 rows

,Neighbourhood ID,Neighbourhood Name
0,138,Eglinton East
1,47,Don Valley Village
2,38,Lansing-Westgate
3,9,Edenbridge-Humber Valley
4,44,Flemingdon Park


In [5]:

toronto_gdf = gpd.read_file(toronto_geo) # Read GEOJSON file to a Geopandas dataframe
toronto_gdf.head() # Display initial dataframe to see what outcome is

NameError: name 'gpd' is not defined

In [ ]:
toronto_gdf = toronto_gdf.iloc[:, 5:]  # Slice dataframe for only relevant attributes
toronto_gdf.rename(columns={'AREA_LONG_CODE': 'Neighbourhood ID'},
                   inplace=True)  # Rename Area_Long_Code as it is same as Neighbourhood ID
toronto_gdf.drop(labels=['AREA_DESC', 'OBJECTID', 'X', 'Y', 'AREA_NAME'],
                 axis=1, inplace=True) # Drop irrelevant columns
toronto_gdf.head() # Display top 5 rows

In [ ]:

toronto_gdf=toronto_gdf.merge(toronto_df,
                              on='Neighbourhood ID') # Use Merge function for both dataset
cols = toronto_gdf.columns.tolist() # Convert column names to a List
cols = cols[-1:] + cols[:-1] # Move last column to first column
toronto_gdf=toronto_gdf[cols] # Reorder the columns in the Geopandas dataframe
toronto_gdf.head() # Display top 5 rows

In [ ]:
df_geo_coor = pd.read_csv("./Geospatial_Coordinates.csv")
df_geo_coor.head()

In [ ]:


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto' # name of city we want the location coordinates
geolocator = Nominatim(user_agent="toronto_explorer") # user agent. You can use any name
location = geolocator.geocode(address)
tor_lat = location.latitude # store longitude value
tor_lon = location.longitude # store latitude value

print(tor_lat,tor_lon )

In [ ]:

MAPBOX_ACCESSTOKEN='pk.eyJ1IjoiZ2xlbm4tbWFjYXBpbmxhYyIsImEiOiJja2tlbjRzdm0wMDIyMnBsZHRsc292M2t4In0.fUlpvGK5iBSf6GfdVtLjrw'

In [ ]:
accesstoken = MAPBOX_ACCESSTOKEN  # Replace with your Mapbox Access Token

# Plotly Express Scatter_Mapbox Initialization
tor_map = px.scatter_mapbox(toronto_gdf, # Geopandas dataframe
                            lat="LATITUDE", # Latitude column in the Geopandas dataframe
                            lon="LONGITUDE", # Longitude column in the Geopandas dataframe
                            hover_name="Neighbourhood Name", # Hover name for the various points
                            hover_data=["Neighbourhood ID"], # Include additional data to hover frame
                            color_discrete_sequence=["blue"], # Colour of data points
                            center={
                                'lat': tor_lat,
                                'lon': tor_lon
                            },
                            zoom=9, # Initial Zoom size of plot
                            height=400,
                            title="Map of Toronto and its 140 Neighbourhoods")
tor_map.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 0}) # set margins of plot
tor_map.update_layout(mapbox_style="streets",
                      mapbox=dict(bearing=-15, accesstoken=MAPBOX_ACCESSTOKEN))

tor_map.show() # Render map

In [ ]:
# Remove the '#' sign below and enter your credentials from Foursquare website
# CLIENT_ID = '   ', CLIENT_SECRET = '  ', LIMIT = '   '

In [ ]:
CLIENT_ID = 'VW005C34AHNMGGTRMVQ4PNLCHN2MS2RHK45DYO40XDC35WES' # Foursquare ID
CLIENT_SECRET = 'FKUTIWGMFHRDLE21410RB15WAUQEN3JELAQVFIEYAR2LV04Y' # Foursquare Secret Key
VERSION = '20190425' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [ ]:
neighborhood_name = toronto_gdf.loc[0, 'Neighbourhood Name']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

In [ ]:
neighborhood_latitude = toronto_gdf.loc[0, 'LATITUDE'] # neighborhood latitude value
neighborhood_longitude =toronto_gdf.loc[0, 'LONGITUDE'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_gdf = getNearbyVenues(names=toronto_gdf['Neighbourhood Name'],
                                   latitudes=toronto_gdf['LATITUDE'],
                                   longitudes=toronto_gdf['LONGITUDE']
                                  )

In [ ]:
toronto_gdf.head()

In [ ]:
toronto_gdf.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_gdf['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_gdf_onehot = pd.get_dummies(toronto_gdf[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_gdf_onehot['Neighborhood'] = toronto_gdf['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_gdf_onehot.columns[-1]] + list(toronto_gdf_onehot.columns[:-1])
toronto_gdf_onehot = toronto_gdf_onehot[fixed_columns]

toronto_gdf_onehot.head()

In [ ]:
toronto_gdf_grouped = toronto_gdf_onehot.groupby('Neighborhood').mean().reset_index()
toronto_gdf_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_gdf_grouped['Neighborhood']

for ind in np.arange(toronto_gdf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_gdf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_gdf_grouped_clustering = toronto_gdf_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_gdf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_gdf_merged = toronto_gdf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_gdf_merged = toronto_gdf_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_gdf_merged.head() # check the last columns!

In [ ]:
# Use the function created above to obtain Venues and Venue Catgories for each Neighbourhood
toronto_venues = getNearbyVenues(names=toronto_gdf['Neighbourhood Name'],
                                 latitudes=toronto_gdf['LATITUDE'],
                                 longitudes=toronto_gdf['LONGITUDE'])
toronto_venues.head()

In [ ]:
import json, requests
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='VW005C34AHNMGGTRMVQ4PNLCHN2MS2RHK45DYO40XDC35WES',
client_secret='FKUTIWGMFHRDLE21410RB15WAUQEN3JELAQVFIEYAR2LV04Y',
v='20190425',
ll='43.6534817, -79.3839347',
query='venue',
limit=10
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [ ]:
data
